In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer
import math
from matplotlib import pyplot as plt
import random
from scipy.optimize import minimize
from qiskit.circuit.library import RZZGate
import networkx as nx
import pandas as pd
import seaborn as sns
import random
from qiskit.quantum_info import Statevector

In [2]:
N = 4
def initialization_state_0(circuit):
    return circuit

def initialization_state_1(circuit):
    circuit.x(0)
    return circuit

def initialization_state_2(circuit):
    circuit.x(1)
    return circuit

def initialization_state_3(circuit):
    circuit.x(0)
    circuit.x(1)
    return circuit

def random_variational_state(circuit):
    for qubit in circuit.qubits:
        circuit.rx(random_theta,qubit)
    return circuit

def rotational_unitary_layer(circuit, gamma, alpha):
    for qubit in circuit.qubits:
        circuit.ry(alpha,qubit)
    for qubit in circuit.qubits:
        circuit.rz(alpha,qubit)
    circuit.barrier()
   
    return circuit

def entanglement_layer(circuit):
    circuit.cx(0,1)
    circuit.cx(1,2)
    circuit.cx(2,3)
    circuit.cx(0,3)
    circuit.barrier()
    return circuit

def anasatz(params, init_state):
    gammas, alphas = create_params(params)
    q = QuantumRegister(N)
    c = ClassicalRegister(N)
    circuit = QuantumCircuit(q,c)
    if init_state == 0:
        circuit = initialization_state_0(circuit)
    if init_state == 1:
        circuit = initialization_state_1(circuit)
    if init_state == 2:
        circuit = initialization_state_2(circuit)
    if init_state == 3:
        circuit = initialization_state_3(circuit)
    if init_state == "random variational state":
        circuit = random_variational_state(circuit)
        init_state = check_state(circuit) #Check if the state generated is equal to state 0, 1, 2 or 3
    
    circuit.barrier()    
    l = int(len(gammas))
    for i in range(l):
        circuit = rotational_unitary_layer(circuit,gammas[i], alphas[i])
        circuit = entanglement_layer(circuit)  #Adding 4 layers to the circuit as we have defined depth as 4
    circuit = rotational_unitary_layer(circuit,gammas[i], alphas[i])    
    circuit.measure(circuit.qubits,c)
    
    simulator = BasicAer.get_backend('qasm_simulator')
    result = execute(circuit, simulator).result()
    counts = result.get_counts(circuit)
    results = counts.keys()
    new_res=[]
    for key in results:
        hold=[]
        for i in range(0,len(key)):
            hold.append(int(key[i]))
        for i in range(0,counts[key]):
            new_res.append(hold)
    return circuit,counts,new_res

def create_params(params):
    alphas = []
    gammas = []
    for i in range(int(len(params)/2)):
        gammas.append(params[2*i])
        alphas.append(params[2*i+1])
    return gammas , alphas
def first_guess_linear(n_layers, m1 = 0.5, m2 = 0.5):
    theta = np.zeros([2*n_layers])
    for i in range(2*n_layers):
        if i%2 == 0:
            theta[i] = m1*(i+1) / (2*n_layers)
        else:
            theta[i] = m2*(2*n_layers-i) / (2*n_layers)
    return (theta)
def binary_to_decimal(bitstring):
    dec = 0
    for i in range(len(bitstring)):
        dec = dec+ ((bitstring[i])*(2**(len(bitstring)-1-i)))
    return dec

def get_maximum_z(counts):
    maximum = 0
    z = ''
    for key in counts.keys():
        if counts[key] > maximum:
            maximum = counts[key]
            z = key 
    return z

def array_of_key(key):
    hold=[]
    for i in range(0,len(key)):
        hold.append(int(key[i]))
    return hold

def check_state(circuit):
    
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)
    circuit = initialization_state_3(circuit)
    
    sv = Statevector(circuit)
    
    q_0 = QuantumRegister(4)
    c_0 = ClassicalRegister(4)
    circuit_0 = QuantumCircuit(q,c)
    circuit_0 = initialization_state_0(circuit_0)
    sv_0 = Statevector(circuit_0)
    
    q_1 = QuantumRegister(4)
    c_1 = ClassicalRegister(4)
    circuit_1 = QuantumCircuit(q,c)
    circuit_0 = initialization_state_1(circuit_1)
    sv_1 = Statevector(circuit_1)
    
    q_2 = QuantumRegister(4)
    c_2 = ClassicalRegister(4)
    circuit_2 = QuantumCircuit(q,c)
    circuit_2 = initialization_state_2(circuit_2)
    sv_2 = Statevector(circuit_2)
    
    q_3 = QuantumRegister(4)
    c_3 = ClassicalRegister(4)
    circuit_3 = QuantumCircuit(q,c)
    circuit_3 = initialization_state_3(circuit_3)
    sv_3 = Statevector(circuit_3)
    
    in_state = "random variational state"

    if sv == sv_0:
        in_state = 0
    if sv == sv_1:
        in_state = 1
    if sv == sv_2:
        in_state = 2
    if sv == sv_3:
        in_state = 3
        
    return in_state

In [3]:
costs = []
def collect_costs(cost, costs):
    costs.append(cost)
def cost_function(params):
    init_state = in_state
    circuit,counts,result = anasatz(params, init_state)
    if init_state == 0:
        output = "1100"
    if init_state == 1:
        output = "0101"
    if init_state == 2:
        output = "1010"
    if init_state == 3:
        output = "0011"
    if init_state == "random variational state":
        output = "0000"
    
    out_bit = array_of_key(output)
    #print("Expected Output State:", out_bit)
    total_cost = 0
    for i in result:
        cost = 0
        for j in range(len(i)):
            cost = cost + 1000*((out_bit[j] - i[j])**2)
        total_cost = total_cost + cost
    
    total_cost = total_cost/1024
    
    collect_costs(total_cost, costs)
    return total_cost

In [4]:
def get_results(input_state, p):
    costs = []
    if input_state == 0:
        expout = "1100"
    if input_state == 1:
        expout = "0101"
    if input_state == 2:
        expout = "1010"
    if input_state == 3:
        expout = "0011"
    if input_state == "random variational state":
        expout = "0000"
    
    #p = 7
   

    init_params = first_guess_linear(p, m1 = 0.5, m2 = 0.5)
    num_iters = 200
    print("optimization for depth :{}".format(p))
    method = 'COBYLA'
    backend="qasm"
    print("initial guess is:" +str(init_params))
    

    out = minimize(cost_function, x0=init_params, method="COBYLA", options={'maxiter':num_iters})
    print("\n")
    print(f'Out: {out}')
    circuit,counts,result = anasatz(out.x, in_state)

    expfrequency = 0
    for key in counts.keys():
        if key == expout:
            expfrequency = counts[key]
    
    z = get_maximum_z(counts)
    #print("Initial state is:", in_state)
    print("Expected output is:", expout)
    print("Frequency of exepected output is:", expfrequency)
    print("Output with maximum frequency, z is:", z)
    print("Frequency of z is:", counts[z])
    
    data = pd.DataFrame([ [init_params], [backend], [out.x],[out.fun],
                        [p], [method],[z], [counts[z]],  [expout], [expfrequency]]) 
            
    data = data.T
    data.columns=[["inital angles" , "backend" , "optimal angles", "cost",
                       "p", "optimizer", "max-bitstring","frequency of max output","expected output","frequency of expected output"]]
    
    return circuit, data

In [6]:
data_final_3 = pd.DataFrame(columns=[["inital angles" , "backend" , "optimal angles", "cost",
                       "p", "optimizer", "max-bitstring","frequency of max output","expected output","frequency of expected output"]])
in_state = 3
print("Initialization state:",3)
for l in range(1,8):
    
    circuit, data = get_results(in_state, l)
    print("---------------------------------------------------------------------------------------------------------------------")
    data_final_3 = data_final_3.append(data)

Initialization state: 3
optimization for depth :1
initial guess is:[0.25 0.25]


Out:      fun: 1006.8359375
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 35
  status: 1
 success: True
       x: array([ 3.29787464, -3.18981953])
Expected output is: 0011
Frequency of exepected output is: 0
Output with maximum frequency, z is: 1011
Frequency of z is: 1020
---------------------------------------------------------------------------------------------------------------------
optimization for depth :2
initial guess is:[0.125 0.375 0.375 0.125]


Out:      fun: 1391.6015625
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 50
  status: 1
 success: True
       x: array([ 1.19145319, -0.2501436 ,  0.54977794,  0.7803031 ])
Expected output is: 0011
Frequency of exepected output is: 218
Output with maximum frequency, z is: 0011
Frequency of z is: 218
-------------------------------------------------------------------------------------------------

In [7]:
data_final_3

,inital angles,backend,optimal angles,cost,p,optimizer,max-bitstring,frequency of max output,expected output,frequency of expected output
0,"[0.25, 0.25]",qasm,"[3.297874643705433, -3.189819527244218]",1006.835938,1,COBYLA,1011,1020,0011,0
0,"[0.125, 0.375, 0.375, 0.125]",qasm,"[1.1914531909791364, -0.25014359725868135, 0.5...",1391.601562,2,COBYLA,0011,218,0011,218
0,"[0.08333333333333333, 0.4166666666666667, 0.25...",qasm,"[1.0753533683067007, 1.4130821932881812, 1.501...",1911.132812,3,COBYLA,1101,133,0011,105
0,"[0.0625, 0.4375, 0.1875, 0.3125, 0.3125, 0.187...",qasm,"[0.20082201819980985, 0.051998963189402995, 1....",6.835938,4,COBYLA,0011,1017,0011,1017
0,"[0.05, 0.45, 0.15, 0.35, 0.25, 0.25, 0.35, 0.1...",qasm,"[-0.23176275453368791, 1.0402746632550297, -0....",1660.15625,5,COBYLA,0001,292,0011,70
0,"[0.041666666666666664, 0.4583333333333333, 0.1...",qasm,"[0.039598536008892474, 0.554474624509436, 0.16...",1441.40625,6,COBYLA,0011,305,0011,305
0,"[0.03571428571428571, 0.4642857142857143, 0.10...",qasm,"[0.07252161270922751, 1.5221177043222829, 0.58...",1371.09375,7,COBYLA,0010,207,0011,75


In [8]:
data_final_2 = pd.DataFrame(columns=[["inital angles" , "backend" , "optimal angles", "cost",
                       "p", "optimizer", "max-bitstring","frequency of max output","expected output","frequency of expected output"]])
in_state = 2
print("Initialization state:",in_state)
for l in range(1,8):
    
    circuit, data = get_results(in_state, l)
    print("---------------------------------------------------------------------------------------------------------------------")
    data_final_2 = data_final_2.append(data)

Initialization state: 2
optimization for depth :1
initial guess is:[0.25 0.25]


Out:      fun: 1000.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 21
  status: 1
 success: True
       x: array([1.38433971, 0.01325682])
Expected output is: 1010
Frequency of exepected output is: 1
Output with maximum frequency, z is: 1110
Frequency of z is: 1022
---------------------------------------------------------------------------------------------------------------------
optimization for depth :2
initial guess is:[0.125 0.375 0.375 0.125]


Out:      fun: 2.9296875
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 47
  status: 1
 success: True
       x: array([ 1.12004789, -0.02760589,  0.34349594, -0.00851551])
Expected output is: 1010
Frequency of exepected output is: 1024
Output with maximum frequency, z is: 1010
Frequency of z is: 1024
----------------------------------------------------------------------------------------------------------

In [9]:
data_final_2

,inital angles,backend,optimal angles,cost,p,optimizer,max-bitstring,frequency of max output,expected output,frequency of expected output
0,"[0.25, 0.25]",qasm,"[1.3843397072205603, 0.013256816638564345]",1000.0,1,COBYLA,1110,1022,1010,1
0,"[0.125, 0.375, 0.375, 0.125]",qasm,"[1.1200478889544423, -0.027605885834554127, 0....",2.929688,2,COBYLA,1010,1024,1010,1024
0,"[0.08333333333333333, 0.4166666666666667, 0.25...",qasm,"[1.0921339248879165, 0.4108839685976134, 0.311...",1897.460938,3,COBYLA,1100,173,1010,26
0,"[0.0625, 0.4375, 0.1875, 0.3125, 0.3125, 0.187...",qasm,"[0.049820383140723824, 0.0919864520439996, -0....",1008.789062,4,COBYLA,0010,1016,1010,0
0,"[0.05, 0.45, 0.15, 0.35, 0.25, 0.25, 0.35, 0.1...",qasm,"[1.3158416734444909, -0.23222720561445315, 0.1...",1060.546875,5,COBYLA,1110,637,1010,69
0,"[0.041666666666666664, 0.4583333333333333, 0.1...",qasm,"[0.1587137866704425, 0.020972075503029923, 1.2...",2.929688,6,COBYLA,1010,1022,1010,1022
0,"[0.03571428571428571, 0.4642857142857143, 0.10...",qasm,"[0.7722612192580552, 0.2520991118172232, 1.323...",1353.515625,7,COBYLA,1010,347,1010,347


In [10]:
data_final_1 = pd.DataFrame(columns=[["inital angles" , "backend" , "optimal angles", "cost",
                       "p", "optimizer", "max-bitstring","frequency of max output","expected output","frequency of expected output"]])
in_state = 1
print("Initialization state:",in_state)
for l in range(1,8):
    
    circuit, data = get_results(in_state, l)
    print("---------------------------------------------------------------------------------------------------------------------")
    data_final_1 = data_final_1.append(data)

Initialization state: 1
optimization for depth :1
initial guess is:[0.25 0.25]


Out:      fun: 1000.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 22
  status: 1
 success: True
       x: array([0.20393259, 0.00515381])
Expected output is: 0101
Frequency of exepected output is: 0
Output with maximum frequency, z is: 0111
Frequency of z is: 1024
---------------------------------------------------------------------------------------------------------------------
optimization for depth :2
initial guess is:[0.125 0.375 0.375 0.125]


Out:      fun: 0.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 52
  status: 1
 success: True
       x: array([ 1.23378723,  0.0023938 ,  0.43574463, -0.01592059])
Expected output is: 0101
Frequency of exepected output is: 1022
Output with maximum frequency, z is: 0101
Frequency of z is: 1022
----------------------------------------------------------------------------------------------------------------

In [11]:
data_final_1

,inital angles,backend,optimal angles,cost,p,optimizer,max-bitstring,frequency of max output,expected output,frequency of expected output
0,"[0.25, 0.25]",qasm,"[0.20393259461585328, 0.005153811248868517]",1000.0,1,COBYLA,0111,1024,0101,0
0,"[0.125, 0.375, 0.375, 0.125]",qasm,"[1.2337872261925693, 0.0023938020351555534, 0....",0.0,2,COBYLA,0101,1022,0101,1022
0,"[0.08333333333333333, 0.4166666666666667, 0.25...",qasm,"[0.07542001981684406, 1.4332295670131427, 1.25...",1948.242188,3,COBYLA,0011,239,0101,21
0,"[0.0625, 0.4375, 0.1875, 0.3125, 0.3125, 0.187...",qasm,"[0.04495278064693296, 0.2571323191717442, -0.0...",1080.078125,4,COBYLA,0001,904,0101,8
0,"[0.05, 0.45, 0.15, 0.35, 0.25, 0.25, 0.35, 0.1...",qasm,"[1.0135997669514896, 0.12611550957283324, 1.41...",1002.929688,5,COBYLA,0111,1019,0101,0
0,"[0.041666666666666664, 0.4583333333333333, 0.1...",qasm,"[1.2582327265143691, -0.09965678781061721, 0.1...",93.75,6,COBYLA,0101,972,0101,972
0,"[0.03571428571428571, 0.4642857142857143, 0.10...",qasm,"[0.8452685246852613, 1.6706020942793303, 0.194...",615.234375,7,COBYLA,0101,606,0101,606


In [12]:
data_final_0 = pd.DataFrame(columns=[["inital angles" , "backend" , "optimal angles", "cost",
                       "p", "optimizer", "max-bitstring","frequency of max output","expected output","frequency of expected output"]])
in_state = 0
print("Initialization state:",in_state)
for l in range(1,8):
    
    circuit, data = get_results(in_state, l)
    print("---------------------------------------------------------------------------------------------------------------------")
    data_final_0 = data_final_0.append(data)

Initialization state: 0
optimization for depth :1
initial guess is:[0.25 0.25]


Out:      fun: 1705.078125
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 30
  status: 1
 success: True
       x: array([1.63429322, 0.81119856])
Expected output is: 1100
Frequency of exepected output is: 132
Output with maximum frequency, z is: 0000
Frequency of z is: 238
---------------------------------------------------------------------------------------------------------------------
optimization for depth :2
initial guess is:[0.125 0.375 0.375 0.125]


Out:      fun: 1725.5859375
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 50
  status: 1
 success: True
       x: array([ 1.12732505, -0.05775529,  0.68473881,  0.92108784])
Expected output is: 1100
Frequency of exepected output is: 117
Output with maximum frequency, z is: 1111
Frequency of z is: 238
---------------------------------------------------------------------------------------------------

In [13]:
data_final_0

,inital angles,backend,optimal angles,cost,p,optimizer,max-bitstring,frequency of max output,expected output,frequency of expected output
0,"[0.25, 0.25]",qasm,"[1.6342932235245105, 0.811198560760694]",1705.078125,1,COBYLA,0000,238,1100,132
0,"[0.125, 0.375, 0.375, 0.125]",qasm,"[1.127325052835474, -0.057755288515910434, 0.6...",1725.585938,2,COBYLA,1111,238,1100,117
0,"[0.08333333333333333, 0.4166666666666667, 0.25...",qasm,"[0.07165116418272588, 0.19209664653353611, 0.9...",1525.390625,3,COBYLA,0000,260,1100,140
0,"[0.0625, 0.4375, 0.1875, 0.3125, 0.3125, 0.187...",qasm,"[-0.09428824918771213, 0.05266151188684762, 0....",1474.609375,4,COBYLA,1000,365,1100,74
0,"[0.05, 0.45, 0.15, 0.35, 0.25, 0.25, 0.35, 0.1...",qasm,"[0.9549223314547601, -0.6559654500824319, -0.1...",1188.476562,5,COBYLA,1000,318,1100,124
0,"[0.041666666666666664, 0.4583333333333333, 0.1...",qasm,"[-0.1614031346345072, 0.9743782246641673, -0.0...",1344.726562,6,COBYLA,1101,308,1100,36
0,"[0.03571428571428571, 0.4642857142857143, 0.10...",qasm,"[1.035700456425274, 0.4642834726098694, 0.1071...",1634.765625,7,COBYLA,0000,447,1100,80


In [19]:
data_final_random = pd.DataFrame(columns=[["inital angles" , "backend" , "optimal angles", "cost",
                       "p", "optimizer", "max-bitstring","frequency of max output","expected output","frequency of expected output"]])
in_state = "random variational state"
random_theta = random.uniform(-np.pi, np.pi)
print("random theta:", random_theta)
print("Initialization state:",in_state)
for l in range(1,8):
    
    circuit, data = get_results(in_state, l)
    print("---------------------------------------------------------------------------------------------------------------------")
    data_final_random = data_final_random.append(data)

random theta: -2.509684258424889
Initialization state: random variational state
optimization for depth :1
initial guess is:[0.25 0.25]


Out:      fun: 1975.5859375
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 28
  status: 1
 success: True
       x: array([1.76657699, 1.80470916])
Expected output is: 0000
Frequency of exepected output is: 108
Output with maximum frequency, z is: 0110
Frequency of z is: 178
---------------------------------------------------------------------------------------------------------------------
optimization for depth :2
initial guess is:[0.125 0.375 0.375 0.125]


Out:      fun: 1362.3046875
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 51
  status: 1
 success: True
       x: array([ 0.18554108,  1.57903128, -0.08901516,  0.92871046])
Expected output is: 0000
Frequency of exepected output is: 230
Output with maximum frequency, z is: 0000
Frequency of z is: 230
------------------------------------------

In [20]:
data_final_random

,inital angles,backend,optimal angles,cost,p,optimizer,max-bitstring,frequency of max output,expected output,frequency of expected output
0,"[0.25, 0.25]",qasm,"[1.7665769882148197, 1.8047091564037794]",1975.585938,1,COBYLA,0110,178,0000,108
0,"[0.125, 0.375, 0.375, 0.125]",qasm,"[0.18554108081062654, 1.579031280635387, -0.08...",1362.304688,2,COBYLA,0000,230,0000,230
0,"[0.08333333333333333, 0.4166666666666667, 0.25...",qasm,"[0.0962615406588667, 0.40315817718760766, 0.22...",1956.054688,3,COBYLA,1001,486,0000,0
0,"[0.0625, 0.4375, 0.1875, 0.3125, 0.3125, 0.187...",qasm,"[-0.017703487484642306, 1.6710862368092965, 1....",1473.632812,4,COBYLA,0000,198,0000,198
0,"[0.05, 0.45, 0.15, 0.35, 0.25, 0.25, 0.35, 0.1...",qasm,"[0.100131818900925, 1.547813517757278, -0.1184...",1428.710938,5,COBYLA,0100,261,0000,98
0,"[0.041666666666666664, 0.4583333333333333, 0.1...",qasm,"[1.1002657223750345, 1.0353245422873871, 1.130...",1149.414062,6,COBYLA,0010,724,0000,70
0,"[0.03571428571428571, 0.4642857142857143, 0.10...",qasm,"[0.7754238407976529, 1.0312210348248319, 1.027...",1285.15625,7,COBYLA,0100,674,0000,22
